Symbolic Regression


In [226]:
import copy
import random
import logging
import os

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

%matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np

Using matplotlib backend: TkAgg


In [205]:
def progn(*args):
    for arg in args:
        arg()


def prog2(out1, out2):
    return partial(progn, out1, out2)


def prog3(out1, out2, out3):
    return partial(progn, out1, out2, out3)


In [206]:
def if_then_else(condition, out1, out2):
    out1() if condition() else out2()


In [207]:
class AntSimulator(object):
    direction = ["north", "east", "south", "west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]

    def __init__(self, max_moves):
        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.routine = None
        self.logging = False
        self.move_history = []

    def _reset(self):
        self.row = self.row_start
        self.col = self.col_start
        self.dir = 1
        self.moves = 0
        self.move_history = []
        self.eaten = 0
        self.matrix_exc = copy.deepcopy(self.matrix)

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])

    def turn_left(self):
        self.move_history.append("LEFT") if self.logging else None
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        self.move_history.append("RIGHT") if self.logging else None
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir + 1) % 4

    def move_forward(self):
        self.move_history.append("FORWARD") if self.logging else None
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col
        return self.matrix_exc[ahead_row][ahead_col] == "food"

    def sense_sides(self):
        left_row = (self.row + self.dir_row[(self.dir - 1) % 4]) % self.matrix_row
        left_col = (self.col + self.dir_col[(self.dir - 1) % 4]) % self.matrix_col
        right_row = (self.row + self.dir_row[(self.dir + 1) % 4]) % self.matrix_row
        right_col = (self.col + self.dir_col[(self.dir + 1) % 4]) % self.matrix_col
        return (self.matrix_exc[left_row][left_col] == "food") or (self.matrix_exc[right_row][right_col] == "food")

    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food, out1, out2)

    def if_food_aside(self, out1, out2):
        return partial(if_then_else, self.sense_sides, out1, out2)

    def run(self, routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()

    def parse_matrix(self, matrix):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)


In [208]:
ant = AntSimulator(200)
map_path = "santa_fe.txt"

In [209]:
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)
pset.addPrimitive(ant.if_food_aside, 2)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addTerminal(ant.move_forward)
pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)

In [210]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [211]:
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [212]:
def set_map(path):
    with  open(path) as trail_file:
        ant.parse_matrix(trail_file)


def rotate_180(ant):
    for list in ant.matrix:
        list.reverse()


def rotate_90(ant):
    ant.matrix.reverse()

    
def evalArtificialAnt(individual):
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    ant.run(routine)
    return ant.eaten,

In [213]:
toolbox.register("evaluate", evalArtificialAnt)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)


In [214]:
def run_evolution():
    random.seed()
    set_map(map_path)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.4, 40, stats, halloffame=hof, verbose=True)

    return pop, hof, stats

In [215]:
pop, hof, stats = run_evolution()

gen	nevals	avg 	min	max
0  	300   	3.22	0  	38 
1  	216   	8.78667	0  	38 
2  	211   	15.3667	0  	38 
3  	214   	21.1833	0  	44 
4  	222   	22.24  	0  	89 
5  	208   	26.52  	0  	89 
6  	221   	28.7667	0  	89 
7  	188   	34.8667	0  	89 
8  	214   	43.3033	0  	89 
9  	217   	48.9833	0  	89 
10 	220   	55.6967	0  	89 
11 	200   	55.82  	0  	89 
12 	214   	56.7333	0  	89 
13 	208   	58.5033	0  	89 
14 	206   	61.26  	0  	89 
15 	197   	64.49  	0  	89 
16 	217   	62.23  	0  	89 
17 	211   	64.8967	0  	89 
18 	208   	65.6767	0  	89 
19 	210   	67.7467	0  	89 
20 	195   	70.3033	0  	89 
21 	212   	68.9133	0  	89 
22 	213   	72.32  	0  	89 
23 	198   	75.8767	0  	89 
24 	220   	71.64  	0  	89 
25 	214   	75.83  	0  	89 
26 	223   	72.3467	0  	89 
27 	227   	74.3733	0  	89 
28 	185   	80.2733	0  	89 
29 	207   	75.24  	0  	89 
30 	210   	72.6233	0  	89 
31 	199   	77.84  	0  	89 
32 	220   	77.3167	0  	89 
33 	221   	77.06  	0  	89 
34 	207   	77.97  	0  	89 
35 	218   	76.2533	0  	89 
36 	213

In [216]:
ant.logging = True
best_ind = hof[0]

In [217]:
best_score = evalArtificialAnt(best_ind)
move_history = ant.move_history

print(len(move_history))

200


In [227]:
dir_row = [1, 0, -1, 0]
dir_col = [0, 1, 0, -1]

def convert_matrix(file):
    with  open(file) as trail_file:
        x = y = direction = -1
        data = list()
        for i, line in enumerate(trail_file):
            data.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    data[-1].append(15)
                elif col == ".":
                    data[-1].append(5)
                elif col == "S":
                    data[-1].append(35)
                    y = i
                    x = j
        dim_y = len(data)
        dim_x = len(data[0])
        return data, x, y, dim_x, dim_y


def turn_left(dir):
    return (dir - 1) % 4


def turn_right(dir):
    return (dir + 1) % 4


def move(data, x, y, dim_x, dim_y, direction):
    data[x][y] = 25
    x = (x + dir_row[direction]) % dim_x
    y = (y + dir_col[direction]) % dim_y
    data[x][y] = 35
    return data, x, y


def parse_movement(data, moves, img, x, y):
        interval = 0.0001
        direction = 1
        dim_y = len(data)
        dim_x = len(data[0])
        for single in moves:
            if single == "LEFT":
                direction = turn_left(direction)
            elif single == "RIGHT":
                direction = turn_right(direction)
            elif single == "FORWARD":
                data, x, y = move(data, x, y, dim_x, dim_y, direction)
                plt.pause(interval)
            img.set_data(data)


In [228]:
# create discrete colormap
cmap = colors.ListedColormap(['white', 'red', 'blue', 'black'])

# 5 - blank | 15 - food | 25 visited | 35 - actual
bounds = [0, 10, 20, 30, 40]
norm = colors.BoundaryNorm(bounds, cmap.N)

data, x, y, dim_x, dim_y = convert_matrix(map_path)

fig, ax = plt.subplots()
img = ax.imshow(data, cmap=cmap, norm=norm)

# draw gridlines
ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
ax.set_xticks(np.arange(0.5, dim_x, 1))
ax.set_yticks(np.arange(0.5, dim_y, 1))

plt.ion()
parse_movement(data, move_history, img, x, y)